In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["groq_api_key"] = os.getenv("groq_api_key")

In [4]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="qwen-2.5-32b")

In [5]:
llm.invoke("hii")

AIMessage(content="Hello! How can I assist you today? Feel free to ask about anything you're curious about or need help with.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 31, 'total_tokens': 56, 'completion_time': 0.063770911, 'prompt_time': 0.003910845, 'queue_time': 0.02197085, 'total_time': 0.067681756}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_7b9ecfcd5f', 'finish_reason': 'stop', 'logprobs': None}, id='run-266d6baa-10c9-4267-b587-5b51e29b49f6-0', usage_metadata={'input_tokens': 31, 'output_tokens': 25, 'total_tokens': 56})

In [6]:
from langchain_community.document_loaders import PyPDFLoader

In [7]:
file_path = r"C:\Users\ROHIT\OneDrive\Desktop\PN\RohitVyavahare_DataScientist(AI_Engineer)_Resume.pdf"
loader = PyPDFLoader(file_path)
docs=loader.load()
len(docs)

2

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
splits=text_splitter.split_documents(docs)

In [10]:
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\Users\ROHIT\anaconda3\envs\genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams


In [12]:
client = QdrantClient(":memory:")

client.create_collection(
    collection_name="demo_collection",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

True

In [13]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embeddings
)

In [14]:
vector_store.add_documents(documents=splits)

['446eb7db34ae42a288dc983bbb1ade96',
 '770293d19bd848f5a5e159f9dbd2cfa9',
 '61d0bc9575924bf58467616deef8cb42',
 '14719f9e3b2f4d07adee244539f28815',
 '286b3803f11a4090a2948be22143ec53',
 '62fd8bbfcce0408e9066e306dadb8d98',
 'f3dade7f35c642918bedae82dad33588']

In [15]:
retreiver=vector_store.as_retriever()
retreiver.invoke("skills")

[Document(metadata={'source': 'C:\\Users\\ROHIT\\OneDrive\\Desktop\\PN\\RohitVyavahare_DataScientist(AI_Engineer)_Resume.pdf', 'page': 0, '_id': '770293d19bd848f5a5e159f9dbd2cfa9', '_collection_name': 'demo_collection'}, page_content='• Machine Learning: Strong knowledge of supervised and unsupervised learning algorithms, including SVMs, Logistic \nRegression, Linear Regression, and Clustering. Experienced with machine learning libraries such as Scikit-Learn. \n• Deep Learning: Hands-on experience with deep learning techniques including CNNs, RNNs, LSTMs, Encoder-\nDecoder architectures, Transformers, and BERT. Familiar with deep learning frameworks such as TensorFlow, \nKeras, and PyTorch. \n• Natural Language Processing (NLP): Skilled in applying NLP techniques to analyze and extract insights from text \ndata. Proficient in using NLP libraries such as NLTK and working with GPT models for text generation and \nunderstanding. \n• API Development: Proficient in building and deploying AP

In [16]:
results = vector_store.similarity_search(
    "skills", k=2
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* • Machine Learning: Strong knowledge of supervised and unsupervised learning algorithms, including SVMs, Logistic 
Regression, Linear Regression, and Clustering. Experienced with machine learning libraries such as Scikit-Learn. 
• Deep Learning: Hands-on experience with deep learning techniques including CNNs, RNNs, LSTMs, Encoder-
Decoder architectures, Transformers, and BERT. Familiar with deep learning frameworks such as TensorFlow, 
Keras, and PyTorch. 
• Natural Language Processing (NLP): Skilled in applying NLP techniques to analyze and extract insights from text 
data. Proficient in using NLP libraries such as NLTK and working with GPT models for text generation and 
understanding. 
• API Development: Proficient in building and deploying APIs using FLASK & FastAPI 
• Generative AI (GenAI): Experienced with GPT, LLaMA, Gemma, Mistral, and Ollama models. Proficient in Hugging [{'source': 'C:\\Users\\ROHIT\\OneDrive\\Desktop\\PN\\RohitVyavahare_DataScientist(AI_Engineer)_Resume.p

In [17]:
from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
                "You are an assistant for question-answering tasks. "
                "Use the following pieces of retrieved context to answer "
                "the question. If you don't know the answer, say that you "
                "don't know. Use three sentences maximum and keep the "
                "answer concise."
                "{context}")

In [18]:
prompt=ChatPromptTemplate.from_messages(
    [
    ("system", system_prompt),
    ("user" ,"{input}")
])

In [19]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [20]:
qa_chain= create_stuff_documents_chain(llm,prompt)
chain=create_retrieval_chain(retreiver,qa_chain)
result=chain.invoke({"input": "who is rohit"})
result["answer"]

'Rohit Vyavahare is a Data Scientist/AI Engineer with expertise in machine learning, deep learning, NLP, and Generative AI. He holds a Bachelor of Engineering in Civil Engineering from Pune University and has certifications in Full Stack Data Science and Machine Learning.'

In [21]:
print(result["context"][0].metadata)

{'source': 'C:\\Users\\ROHIT\\OneDrive\\Desktop\\PN\\RohitVyavahare_DataScientist(AI_Engineer)_Resume.pdf', 'page': 0, '_id': '446eb7db34ae42a288dc983bbb1ade96', '_collection_name': 'demo_collection'}


In [24]:
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import ConversationalRetrievalChain

# Create memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Create the conversational chain
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retreiver,
    memory=memory,
    verbose=True
)

# Example usage with memory
result = chain.invoke({"question": "who is rohit?"})
print(result["answer"])

# Ask follow-up question
result = chain.invoke({"question": "what are his skills?"})
print(result["answer"])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
ROHIT VYAVAHARE 
Data Scientist/AI Engineer     
rohitvyavahare2001@gmail.com                    Nashik,Maharashtra,India                    +917262959787 
Linkedin       - https://www.linkedin.com/in/rohitvyavahare2001/         GitHub      - https://github.com/RohitVyavahare2001 
    
SUMMARY 
As a Data Scientist/AI Engineer, I specialize in converting complex data into actionable insights to drive business outcomes. 
With expertise in machine learning, deep learning, NLP, and Generative AI, I deliver impactful solutions. Proficient in SQL, 
Python, and advanced data visualization, I leverage my skills in statistics, data mining, and LLMs to communicate findings 
and enable data-driven decisions. 
_